In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
"""Beautiful Soup is a library that makes it easy to scrape information from web pages."""
from bs4 import BeautifulSoup 

# Requests allows you to send HTTP/1.1 requests extremely easily.
import requests   
import time
import csv


In [3]:
url = 'https://www.fifa.com/fifa-world-ranking/ranking-table/men/'
headers = {"User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36'}
page = requests.get(url, headers = headers)

In [4]:
soup = BeautifulSoup(page.content, 'html.parser')
ranking_schedule_dropdown_menu = soup.find('ul' , attrs = {'class': 'fi-ranking-schedule__nav dropdown-menu'}).find_all('li')
len(ranking_schedule_dropdown_menu)

307

In [5]:
ranking_schedule_dropdown_menu[0]  
# first element in the ranking_schedule_dropdown_menu list


<li class="fi-ranking-schedule__nav__item" data-value="id13078">
<a href="/fifa-world-ranking/ranking-table/men/rank/id13078/">22 October 2020</a>
</li>

In [6]:
# We need to extract url from the above <li> element so that we can make requests for 
# any particular date 

#For example,to make a request for 22nd October 2020, the url will be
# url = 'https://www.fifa.com' + ranking_schedule_dropdown_menu[0].find('a').get('href')

#Similarly, for 17th September 2020, the request url will be
# url = 'https://www.fifa.com' + ranking_schedule_dropdown_menu[1].find('a').get('href') and so on.

print(ranking_schedule_dropdown_menu[0].find('a').get('href'))
print(ranking_schedule_dropdown_menu[0].get_text().strip('\n'))

print(ranking_schedule_dropdown_menu[1].find('a').get('href'))
print(ranking_schedule_dropdown_menu[1].get_text().strip('\n'))

print(ranking_schedule_dropdown_menu[2].find('a').get('href'))
print(ranking_schedule_dropdown_menu[2].get_text().strip('\n'))

# So, you can clearly see we need to iterate to construct request url as we try to fetch the table data
# available from 1992 - till date


/fifa-world-ranking/ranking-table/men/rank/id13078/
22 October 2020
/fifa-world-ranking/ranking-table/men/rank/id13043/
17 September 2020
/fifa-world-ranking/ranking-table/men/rank/id12884/
16 July 2020


In [7]:
table = soup.find('table', attrs = {'id':'rank-table'})
table.find_all('th')  # Table Headers

[<th class="fi-table__th fi-table__rank"><div class="fi-table__th__wrap"> <span class="th-text">
     Rnk
   </span>
 <span class="arrows"></span>
 </div></th>,
 <th class="fi-table__th fi-table__teamname"><div class="fi-table__th__wrap"> <span class="th-text">
     Team
   </span>
 <span class="th-text-abbr">
 <abbr title="Team">Team</abbr>
 </span>
 <span class="arrows"></span>
 </div></th>,
 <th class="fi-table__th fi-table__points"><div class="fi-table__th__wrap"> <span class="th-text">
     Total Points
   </span>
 <span class="th-text-abbr">
 <abbr title="Total Points">PTS</abbr>
 </span>
 <span class="arrows"></span>
 </div></th>,
 <th class="fi-table__th fi-table__prevpoints"><div class="fi-table__th__wrap"> <span class="th-text">
     Previous Points
   </span>
 <span class="th-text-abbr">
 <abbr title="Previous Points">Prev.Pts</abbr>
 </span>
 <span class="arrows"></span>
 </div></th>,
 <th class="fi-table__th fi-table__rankingmovement"><div class="fi-table__th__wrap"> <span

In [8]:
rows = []
rows.append(['Rank', 'Team', 'PTS', 'Prev.Pts', 'Change (+/-)', 'Date'])  # Table Headers

In [9]:
# for 22nd October 2020

results = table.find_all('tr')
print(len(results))  # ( 210 Countries Data Rows + 1 Header Row) <-- 211

results[0]  # The very first row contains Table Headers
results[1].find_all('td')  # Table Data
print(results[1].find_all('td')[0].get_text())   # Rank
print(results[1].find_all('div')[2].find_all('span')[0].get_text()) # Team (Country)
print(results[1].find_all('td')[2].get_text())  # PTS
print(results[1].find_all('td')[3].get_text())  # Prev.Pts
print(results[1].find_all('td')[4].get_text())  #Change
print(ranking_schedule_dropdown_menu[0].get_text().strip('\n'))  # Date

211
1
Belgium
1765
1773
0
22 October 2020


In [10]:
for i in range(0, len(ranking_schedule_dropdown_menu)):
    url = 'https://www.fifa.com' + ranking_schedule_dropdown_menu[i].find('a').get('href')
    date = ranking_schedule_dropdown_menu[i].get_text().strip('\n')
    page = requests.get(url, headers = headers)
    table = soup.find('table', attrs = {'id':'rank-table'})
    results = table.find_all('tr')
    for result in results:
        data = result.find_all('td')
        if len(data) == 0:
            continue
        rank = data[0].get_text()
        team = data[1].find_all('div')[2].find_all('span')[0].get_text()
        points = data[2].get_text()
        prev_points = data[3].get_text()
        change = data[4].get_text()
        rows.append([rank, team, points, prev_points, change, date])
# rows


In [11]:
len(rows)

64471

In [12]:

# Writing in a .csv file
with open('fifa_2020_Country_Rankings_Men_1992_2020.csv','w', encoding="utf-8", newline="") as f_output:
    csv_output = csv.writer(f_output)
    csv_output.writerows(rows)